In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from numpy import genfromtxt
from sklearn.preprocessing import LabelEncoder



In [12]:
# Load Data
x = genfromtxt('../Data/WISDM_x.csv', delimiter=',')
y_df = pd.read_csv('../Data/WISDM_y.csv')
y = y_df.values.flatten()  # Flatten if y is 2D

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Function to create time series dataset
def create_series(x, y, timestep, overlap):
    slide_step = int(timestep * (1 - overlap))
    data_num = int((len(x) / slide_step) - 1)
    dataset = np.ndarray(shape=(data_num, timestep, x.shape[1]))
    labels = []

    for i in range(data_num):
        labels.append(y[slide_step * (i + 1) - 1])
        for j in range(timestep):
            dataset[i, j, :] = x[slide_step * i + j, :]

    return dataset, np.array(labels)

# Create time series
timestep = 16  # Replace with your value
overlap = 0.5  # Replace with your value
X_series, y_series = create_series(x, y_encoded, timestep, overlap)

In [15]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_series, y_series, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
x_train_tensor = torch.tensor(X_train, dtype=torch.float32)
x_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)



In [16]:
# Define the MLP model
class MyMLP(nn.Module):
    def __init__(self, input_size, num_classes=6):
        super(MyMLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# Model Initialization
input_size = timestep * X_series.shape[2]  # Calculate input size
model = MyMLP(input_size)

# DataLoader
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)



In [28]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Training function
def train(model, train_loader, criterion, optimizer, epochs=100):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch}, Loss: {total_loss / len(train_loader)}')

# Train the model
train(model, train_loader, criterion, optimizer)

#
model_path = "/Users/sandeep/Desktop/BUCourses/Project/saved_models/Pytorch/MLP_base.pth"
torch.save(model.state_dict(), model_path)


Epoch 0, Loss: 0.16012097318550367
Epoch 1, Loss: 0.15559006132194142
Epoch 2, Loss: 0.15477679119312956
Epoch 3, Loss: 0.15052282262272212
Epoch 4, Loss: 0.14968625443883343
Epoch 5, Loss: 0.14579738807237091
Epoch 6, Loss: 0.14456765272161795
Epoch 7, Loss: 0.14171782453893955
Epoch 8, Loss: 0.14035277218625192
Epoch 9, Loss: 0.13889665605276735
Epoch 10, Loss: 0.13722214659584778
Epoch 11, Loss: 0.13445652944430936
Epoch 12, Loss: 0.13232851816484562
Epoch 13, Loss: 0.13131979368746471
Epoch 14, Loss: 0.12949632033939085
Epoch 15, Loss: 0.12877123934132131
Epoch 16, Loss: 0.12641413919931593
Epoch 17, Loss: 0.12648512614567467
Epoch 18, Loss: 0.12372869237310889
Epoch 19, Loss: 0.12246137335142099
Epoch 20, Loss: 0.12250998188765386
Epoch 21, Loss: 0.11969273903467942
Epoch 22, Loss: 0.11910039810859978
Epoch 23, Loss: 0.11975298379140847
Epoch 24, Loss: 0.11683214956428856
Epoch 25, Loss: 0.11681781145461807
Epoch 26, Loss: 0.11490011842441604
Epoch 27, Loss: 0.11572008957545814
Ep

In [29]:
def evaluate(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            loss = criterion(output, target)
            total_loss += loss.item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'Test set: Average loss: {total_loss / len(test_loader)}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)')

# DataLoader for test set
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# Evaluate the model
evaluate(model, test_loader, criterion)


Test set: Average loss: 0.5581411203599616, Accuracy: 24909/27455 (91%)


In [30]:
import torch
from sklearn.metrics import accuracy_score
import numpy as np
import time
import psutil
from pathlib import Path

def compute_metrics_base(model, x_test, y_test, model_path):
    """
    Compute the accuracy of the PyTorch model.

    :param model: PyTorch model.
    :param x_test: Test dataset features (as a PyTorch Tensor).
    :param y_test: Test dataset labels (as a NumPy array).
    :param model_dir: Directory where the PyTorch model files are stored.
    :return: None
    """

    model.eval()
    with torch.no_grad():
        # Get the model's predictions
        outputs = model(x_test)
        _, predicted_labels = torch.max(outputs, 1)

        # Convert y_test to tensor if it's not already
        true_labels = torch.tensor(y_test) if not isinstance(y_test, torch.Tensor) else y_test
        true_labels = true_labels.squeeze()  # Remove unnecessary dimensions

    model_file = Path(model_path)

    # Size in bytes
    model_size_bytes = model_file.stat().st_size

    # Convert size to kilobytes (optional)
    model_size_kb = model_size_bytes / 1024
    print(f"Size of the model: {model_size_kb:.2f} KB")

    # Compute accuracy
    accuracy = accuracy_score(true_labels.numpy(), predicted_labels.numpy())
    print(f'Accuracy on the test set: {accuracy:.2%}')


In [31]:
def measure_cpu_utilization_and_run(func, *args, **kwargs):
    """
    Measure CPU utilization while running a function.

    Parameters:
        func (function): The function to be executed.
        *args: Arguments to be passed to func.
        **kwargs: Keyword arguments to be passed to func.

    Returns:
        float: CPU utilization percentage during the execution of func.
        float: The elapsed time during the execution of func.
        any: The result of func execution.
    """
    
    # Measure CPU utilization before execution
    cpu_percent_before = psutil.cpu_percent(interval=None)

    # Record the start time
    start_time = time.time()

    # Execute the function and store its result
    result = func(*args, **kwargs)

    # Record the end time
    end_time = time.time()

    # Measure CPU utilization after execution
    cpu_percent_after = psutil.cpu_percent(interval=None)

    # Calculate elapsed time and average CPU utilization
    elapsed_time = end_time - start_time
    average_cpu_utilization = (cpu_percent_before + cpu_percent_after) / 2

    return average_cpu_utilization, elapsed_time, result



In [32]:
# Measure CPU usage and inference time
cpu_usage, inference_time, _ = measure_cpu_utilization_and_run(compute_metrics_base, model, x_test_tensor, y_test_tensor, model_path)

print(f'CPU usage during inference: {cpu_usage:.2f}%')
print(f'Inference time: {inference_time:.4f} seconds')


Size of the model: 60.83 KB
Accuracy on the test set: 90.73%
CPU usage during inference: 40.85%
Inference time: 0.0067 seconds


In [37]:
import torch.quantization
model = MyMLP(input_size)
model.load_state_dict(torch.load(model_path))
model.eval()

MyMLP(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=48, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=6, bias=True)
)

In [39]:
torch.backends.quantized.engine = 'qnnpack'

quantized_model = torch.quantization.quantize_dynamic(
    model,  # the original model
    {nn.Linear},  # a set of layers to dynamically quantize
    dtype=torch.qint8)  # the target dtype for quantized weights


In [42]:
quantized_model_path = "/Users/sandeep/Desktop/BUCourses/Project/saved_models/Pytorch/MLP_Quantized.pth"
torch.save(quantized_model.state_dict(), quantized_model_path)


In [45]:
# Measure CPU usage and inference time
cpu_usage, inference_time, _ = measure_cpu_utilization_and_run(compute_metrics_base, quantized_model, x_test_tensor, y_test_tensor, quantized_model_path)

print(f'CPU usage during inference: {cpu_usage:.2f}%')
print(f'Inference time: {inference_time:.4f} seconds')


Size of the model: 19.58 KB
Accuracy on the test set: 90.76%
CPU usage during inference: 23.10%
Inference time: 0.0280 seconds


In [46]:
def print_sample_predictions(model, x_test, y_test, num_samples=5):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        # Predict on the test set
        outputs = model(x_test)
        _, predicted = torch.max(outputs, 1)

        print("Sample predictions:\n")
        for i in range(num_samples):
            print(f"x_test[{i}]: {x_test[i]}")
            print(f"Actual label (y_test[{i}]): {y_test[i]}")
            print(f"Predicted label: {predicted[i]}")
            print("\n")


In [47]:
# Assuming you're using the first num_samples of x_test and y_test
num_samples = 5
print_sample_predictions(model, x_test_tensor[:num_samples], y_test_tensor[:num_samples], num_samples=5)


Sample predictions:

x_test[0]: tensor([[-19.5700,  13.0600,  -1.3100],
        [-19.5700,  -4.2900,  -3.9100],
        [ -6.5900,  -5.7500,   6.5100],
        [ -0.4600,  -9.5300,   6.0900],
        [  8.3900,  -7.5000,   9.3400],
        [ -5.2400,   6.5100, -10.9900],
        [ -7.5000,   4.6000,   1.8000],
        [-13.5900,   4.3700,  -7.4600],
        [-19.5700,   4.4800,   6.6600],
        [-19.1500,  -3.8300,  -4.1800],
        [-17.8800,   0.1900,  -4.1800],
        [ -5.0900,   9.5800,  15.3200],
        [ -5.7500,   6.9700,  -2.1800],
        [  7.0400,  11.6500,   5.6700],
        [ -2.1800,   1.9200,  -1.5700],
        [-16.4000,   5.7100,   1.5000]])
Actual label (y_test[0]): 1
Predicted label: 1


x_test[1]: tensor([[ 4.8200, 11.2600,  2.7200],
        [ 5.9400, 12.0300,  0.5000],
        [ 5.4300, 13.5300, -3.3400],
        [-0.1100, 12.5700, -5.3300],
        [-2.4500, 13.8700, -0.6900],
        [ 2.1500, 10.4200,  1.0400],
        [ 3.7600,  6.0500, -3.7200],
        